In [217]:
import ccxt
from datetime import datetime
import pandas as pd
from sympy import Limit
import numpy as np



In [260]:

def fetchData(exchange, symbol, timeframe, since="26-04-2022", limit=30):
    exchange_list = [
        "kraken",
        'ftx',
        'currencycom',
        'coinmate' ]

    try:
        exchange = getattr(ccxt, exchange)()

    #En caso de que no se encuentre en ccxt
    except AttributeError:
        print("-" * 36, " ERROR ", "-" * 35)
        print(
            'Plataforma "{}" not encontrada.'.format(
                exchange
            )
        )
        print("Disponibles:")
        print(exchange_list)
        print("-" * 80)
        quit()

    data = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
    header = ["Timestamp", "open", "high", "low", "close", "volume"]
    df = pd.DataFrame(data, columns=header)

    df.Timestamp = (
        df.Timestamp / 1000
    )  
    df["date"] = pd.to_datetime(df.Timestamp, unit="s")
    df = df[["date", "close", "volume"]]
    

    df["open"] = pd.to_numeric(df["open"])
    df["high"] = pd.to_numeric(df["high"])
    df["low"] = pd.to_numeric(df["low"])
    df["close"] = pd.to_numeric(df["close"])
    df["volume"] = pd.to_numeric(df["volume"])

    return df


In [258]:
def fetch_order_book (symbol,limit):
    exchange = ccxt.binance()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    df = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    df['Vwap Ask'] =(np.cumsum(df.Ask*df.AVolume) / np.cumsum(df.AVolume))
    df['Vwap Bid'] =(np.cumsum(df.Bid*df.BVolume) / np.cumsum(df.BVolume))
    df["Total Volume"]=(df.AVolume + df.BVolume)
    df["Midprice"]=((df.Ask + df.Bid)/2)
    df['Vwap Ask'] =(np.cumsum(df.Ask*df.AVolume) / np.cumsum(df.AVolume))
    df['Vwap Bid'] =(np.cumsum(df.Bid*df.BVolume) / np.cumsum(df.BVolume))
    
                       
    return df


In [259]:
print(fetch_order_book('BTC/USDT',30))

         Ask  AVolume       Bid  BVolume      Vwap Ask      Vwap Bid  \
0   39283.38  0.83267  39283.37  6.63415  39283.380000  39283.370000   
1   39283.42  0.00026  39283.27  1.84007  39283.380012  39283.348286   
2   39283.62  0.00039  39282.92  0.00412  39283.380125  39283.348078   
3   39283.65  0.06200  39281.86  0.69444  39283.398813  39283.235421   
4   39283.78  0.01550  39281.77  0.12727  39283.405300  39283.215367   
5   39284.44  0.07935  39281.72  0.23216  39283.488219  39283.178947   
6   39284.91  0.23009  39281.47  0.26482  39283.756307  39283.132753   
7   39284.92  0.12000  39280.66  0.10022  39283.860498  39283.107713   
8   39285.29  0.80000  39280.62  0.00026  39284.394827  39283.107648   
9   39285.58  0.00039  39280.46  0.00039  39284.395042  39283.107544   
10  39286.09  0.22000  39280.23  0.34366  39284.553003  39283.010987   
11  39286.41  0.34366  39280.22  0.00850  39284.788988  39283.008672   
12  39286.45  0.16000  39280.00  0.05092  39284.881772  39282.99